<a href="https://colab.research.google.com/github/DorcasBwire/BwireDorcas/blob/main/constitution_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install dotenv
!pip install langchain_community
!pip install langchain_huggingface
!pip install langchain_chroma
!pip install groq
!pip install langchain_groq

  Using cached groq-0.25.0-py3-none-any.whl.metadata (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 10.6 MB/s eta 0:00:00


In [20]:
#import libraries
import os
from dotenv import load_dotenv
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_groq import ChatGroq
from langchain.document_loaders import PyPDFLoader
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


In [6]:
#Load env
load_dotenv()

False

In [5]:
#Read env. files
os.environ["GROQ_API_KEY"] = "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"


GROQ_API_KEY = os.environ["GROQ_API_KEY"]


In [7]:
# Confirm the retrieval of the API keys
if GROQ_API_KEY:
    print("GROQ_API_KEY is set")
else:
    print("GROQ_API_KEY is not set")

GROQ_API_KEY is set


In [8]:
#Load the PDF file
!pip install PyPDF
loader = PyPDFLoader("constitution_of_kenya.pdf")
documents = loader.load()
print("PDF loaded successfully")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 22.6 MB/s eta 0:00:00
PDF loaded successfully


In [9]:
#Split the document into smaller chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(documents)

In [10]:
print(f"Total chunks: {len(documents)}")


Total chunks: 143


In [11]:
#embedding model
embedding = HuggingFaceEmbeddings(
    model_name="all-mpnet-base-v2",
)

<ipython-input-11-a79308640353>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
#Create a vector store
db = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory="chroma_db"
)
print("Vector store created successfully")

Vector store created successfully


In [14]:
#Define the retiever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

In [16]:
#Define the prompt template
prompt = ChatPromptTemplate.from_messages(
    ["""You are a constitutional analyst specialized in the Kenyan Constitution.
    Use only the following excerpts from the Kenyan Constitution to answer the question.
    If the specific information isn't found in these excerpts, state that the information
isn't available in the provided constitutional sections rather than speculating.

CONSTITUTIONAL EXCERPTS:
{context}

Question: {question}
Answer based strictly on the Kenyan Constitution:"""
    ]
)

In [17]:
#Initialize the model
model = ChatGroq(
    model = "llama-3.3-70b-versatile",
    api_key = GROQ_API_KEY,
    temperature = 0,
    prompt = prompt
)

/usr/local/lib/python3.11/dist-packages/langchain_groq/chat_models.py:364: UserWarning: WARNING! prompt is not default parameter.
                    prompt was transferred to model_kwargs.
                    Please confirm that prompt is what you intended.
  warnings.warn(


In [21]:
#Set up RAG Chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)



In [23]:
#test the retriever
query = "What are the children rights regarding education according to the Kenyan Constitution?"
docs = retriever.get_relevant_documents(query)

# Print results
for i, doc in enumerate(docs):
    print(f"\n--- Document {i+1} ---\n")
    print(doc.page_content)



--- Document 1 ---

Constitution of Kenya  Kenya
53. Children
(1) Every child has the right—
(a) to a name and nationality from birth;
(b) to free and compulsory basic education;
(c) to basic nutrition, shelter and health care;
(d) to be protected from abuse, neglect, harmful cultural practices, all forms of violence,
inhuman treatment and punishment, and hazardous or exploitative labour;
(e) to parental care and protection, which includes equal responsibility of the mother and father
to provide for the child, whether they are married to each other or not; and
(f) not to be detained, except as a measure of last resort, and when detained, to be held—
(i) for the shortest appropriate period of time; and
(ii) separate from adults and in conditions that take account of the child’s sex and age.
(2) A child’s best interests are of paramount importance in every matter concerning the child.
54. Persons with disabilities
(1) A person with any disability is entitled—
(a) to be treated with dign